In [1]:
import pandas as pd
import praw
import datetime        
from time import sleep
import random
import time



In [2]:
import praw
import pandas as pd
from tqdm import tqdm

In [3]:
user_subs=pd.read_csv("user_subreddits.csv")

In [4]:
user_subs

,Username,Subreddit_1,Subreddit_2,Subreddit_3,Subreddit_4,Subreddit_5,Subreddit_6,Subreddit_7,Subreddit_8,Subreddit_9,...,Subreddit_91,Subreddit_92,Subreddit_93,Subreddit_94,Subreddit_95,Subreddit_96,Subreddit_97,Subreddit_98,Subreddit_99,Subreddit_100
0,italianblend,Catholicism,Catholicism,Catholicism,Catholicism,Catholicism,carnivorediet,samsclub,Catholicism,samsclub,...,carnivorediet,Catholicism,samsclub,Catholicism,Catholicism,Catholicism,Catholicism,Catholicism,Catholicism,samsclub
1,Theinternetdumbens,JoeRogan,stalker,stalker,PublicFreakout,stalker,awakened,SteveWallis,SteveWallis,stalker,...,PaymoneyWubby,conspiracy,carnivorediet,carnivorediet,stalker,carnivorediet,carnivorediet,PaymoneyWubby,stalker,stalker
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Maleficent-Sleep9900,SingleAndHappy,PMDDxADHD,PMDDxADHD,carnivorediet,adhdwomen,CPTSD,carnivorediet,sexualassault,longnaturalnails,...,SingleAndHappy,CPTSD,BPD,childfree,longhair,PMDDxADHD,4bmovement,PMDDxADHD,longhair,4bmovement
4,starsofalgonquin,fujix,leafs,canada,nhl,leafs,leafs,leafs,hockey,nhl,...,AnimalCollective,75HARD,sharpening,fountainpens,AnimalCollective,AnimalCollective,AnimalCollective,cinescenes,cinescenes,AnimalCollective
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14876,Unknowndietetics,glioblastoma,glioblastoma,dietetics,dietetics,pitbulls,Residency,AMA,dietetics,caseoh_,...,AMA,AMA,AMA,AMA,AMA,AMA,AMA,AMA,AMA,AMA
14877,pearlyjay26,DietitiansSaidWhatNow,DietitiansSaidWhatNow,DietitiansSaidWhatNow,DietitiansSaidWhatNow,foodhacks,foodhacks,foodhacks,foodhacks,foodhacks,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14878,ComfortablePea5238,ketorecipes,DietitiansSaidWhatNow,texts,RD2B,RD2B,RD2B,RD2B,HealthyFood,HealthyFood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14879,Outrageous_Let7399,RD2B,dietetics,dietetics,RD2B,DietitiansSaidWhatNow,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
comments=pd.read_csv("Comments.csv")

In [7]:
# Count occurrences of each (author, subreddit) pair
author_subreddit_counts = comments.groupby(['author', 'subreddit']).size().reset_index(name='post_count')

# Convert to list of dictionaries
author_subreddit_list = [
    {'author': author, 'subreddits': {sub: count for sub, count in sub_data.values}}
    for author, sub_data in author_subreddit_counts.groupby('author')[['subreddit', 'post_count']]
]
longest_dict = max(author_subreddit_list, key=lambda x: len(x['subreddits']))

In [9]:
# Melt the DataFrame to transform multiple subreddit columns into one column
subreddit_counts = user_subs.melt(id_vars=['Username'], value_name='Subreddit')

# Count occurrences of each subreddit
subreddit_counts = subreddit_counts['Subreddit'].value_counts().reset_index()

# Rename columns for clarity
subreddit_counts.columns = ['Subreddit', 'Count']

In [11]:
# Convert the list of dictionaries into a dictionary for easier merging
subreddit_dict = {d['author']: d['subreddits'] for d in author_subreddit_list}

# Merge with user_df by mapping the "Username" column to the subreddit_dict
user_subs['original_subreddits'] = user_subs['Username'].map(subreddit_dict)


In [12]:
user_subs

,Username,Subreddit_1,Subreddit_2,Subreddit_3,Subreddit_4,Subreddit_5,Subreddit_6,Subreddit_7,Subreddit_8,Subreddit_9,...,Subreddit_92,Subreddit_93,Subreddit_94,Subreddit_95,Subreddit_96,Subreddit_97,Subreddit_98,Subreddit_99,Subreddit_100,original_subreddits
0,italianblend,Catholicism,Catholicism,Catholicism,Catholicism,Catholicism,carnivorediet,samsclub,Catholicism,samsclub,...,Catholicism,samsclub,Catholicism,Catholicism,Catholicism,Catholicism,Catholicism,Catholicism,samsclub,{'carnivorediet': 5}
1,Theinternetdumbens,JoeRogan,stalker,stalker,PublicFreakout,stalker,awakened,SteveWallis,SteveWallis,stalker,...,conspiracy,carnivorediet,carnivorediet,stalker,carnivorediet,carnivorediet,PaymoneyWubby,stalker,stalker,"{'Ketovore': 2, 'carnivorediet': 3}"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Maleficent-Sleep9900,SingleAndHappy,PMDDxADHD,PMDDxADHD,carnivorediet,adhdwomen,CPTSD,carnivorediet,sexualassault,longnaturalnails,...,CPTSD,BPD,childfree,longhair,PMDDxADHD,4bmovement,PMDDxADHD,longhair,4bmovement,"{'carnivore': 3, 'carnivorediet': 19}"
4,starsofalgonquin,fujix,leafs,canada,nhl,leafs,leafs,leafs,hockey,nhl,...,75HARD,sharpening,fountainpens,AnimalCollective,AnimalCollective,AnimalCollective,cinescenes,cinescenes,AnimalCollective,"{'RawMeat': 2, 'carnivorediet': 1}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14876,Unknowndietetics,glioblastoma,glioblastoma,dietetics,dietetics,pitbulls,Residency,AMA,dietetics,caseoh_,...,AMA,AMA,AMA,AMA,AMA,AMA,AMA,AMA,AMA,{'DietitiansSaidWhatNow': 2}
14877,pearlyjay26,DietitiansSaidWhatNow,DietitiansSaidWhatNow,DietitiansSaidWhatNow,DietitiansSaidWhatNow,foodhacks,foodhacks,foodhacks,foodhacks,foodhacks,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'DietitiansSaidWhatNow': 4}
14878,ComfortablePea5238,ketorecipes,DietitiansSaidWhatNow,texts,RD2B,RD2B,RD2B,RD2B,HealthyFood,HealthyFood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'DietitiansSaidWhatNow': 1}
14879,Outrageous_Let7399,RD2B,dietetics,dietetics,RD2B,DietitiansSaidWhatNow,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'DietitiansSaidWhatNow': 1}


In [15]:
len(subreddit_counts)


35291

In [14]:
subreddit_counts[subreddit_counts["Count"]>=4].head(50)

,Subreddit,Count
0,carnivorediet,42764
1,StopEatingSeedOils,13090
2,AskReddit,12972
3,SaturatedFat,8534
4,exvegans,8376
5,AnimalBased,8075
6,carnivore,7214
7,conspiracy,5949
8,AITAH,5644
9,Biohackers,5454
